# ELEC 221 Assignment 6: radio 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from IPython.display import Audio, Image
from IPython.core.display import HTML

In this problem, you'll implement a simulated AM radio system that is a little more realistic in terms of the operation of the receiver and demodulation process. 

Suppose you and your colleague have been put in charge of a radio broadcast company that manages 5 stations at the following frequencies (in kHz): 580, 600, 620, 640, and 660. Note that typically, AM is used for talk radio, which requires a bandwidth of ~10 kHz. However to make things interesting, we're going to use music which requires a wider range.

**Question 0.1 (1 point)**. To make things more meaningful for you (hopefully!), choose five songs, perhaps of different genres, for your radio stations. Select ~4-5s clips of each, and use a sampling rate of somewhere between 40000-48000.

In [ ]:
# Prepare some samples and load them in here.
# input_audio = wavfile.read("my_song.wav") 

# The result of importing a wavfile should be a tuple of two elements:
# the sample rate, and the audio signal, which may be have multiple channels.
# sample_rate = input_audio[0]

# My audio sample had multiple channels. Yours may not.
# In my case, I took just one of these channels, and converted it from 
# integer values to floats.
# channel_0 = input_audio[1][:, 0]
# song_1 = np.array(channel_0, dtype=np.float64) / np.max(channel_0)

In [16]:
input_audio0 = wavfile.read("flashingLights.wav")
input_audio1 = wavfile.read("favorite.wav")
input_audio2 = wavfile.read("newYork.wav")
input_audio3 = wavfile.read("photograph.wav")
input_audio4 = wavfile.read("redroom.wav")

sample_rate = input_audio[0]
sample_rate1 = input_audio1[0]
sample_rate2 = input_audio2[0]
sample_rate3 = input_audio3[0]
sample_rate4 = input_audio4[0]

channel_0 = input_audio[1][:,0]
channel_01 = input_audio1[1][:,0]
channel_02 = input_audio2[1][:,0]
channel_03 = input_audio3[1][:,0]
channel_04 = input_audio4[1][:,0]

song_1 = np.array(channel_0, dtype=np.float64) / np.max(channel_0)
song_1_audio = Audio(song_1, rate= sample_rate)

song_2 = np.array(channel_01, dtype=np.float64) / np.max(channel_01)
song_2_audio = Audio(song_2, rate = sample_rate1)

song_3 = np.array(channel_02, dtype=np.float64) / np.max(channel_02)
song_3_audio = Audio(song_3, rate = sample_rate2)

song_4 = np.array(channel_03, dtype=np.float64) / np.max(channel_03)
song_3_audio = Audio(song_4, rate = sample_rate3)


song_5 = np.array(channel_04, dtype=np.float64) / np.max(channel_04)
song_3_audio = Audio(song_5, rate = sample_rate4)


C:\Users\1mack\AppData\Local\Temp\ipykernel_20872\4255260337.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  input_audio0 = wavfile.read("flashingLights.wav")
C:\Users\1mack\AppData\Local\Temp\ipykernel_20872\4255260337.py:2: WavFileWarning: Chunk (non-data) not understood, skipping it.
  input_audio1 = wavfile.read("favorite.wav")
C:\Users\1mack\AppData\Local\Temp\ipykernel_20872\4255260337.py:3: WavFileWarning: Chunk (non-data) not understood, skipping it.
  input_audio2 = wavfile.read("newYork.wav")
C:\Users\1mack\AppData\Local\Temp\ipykernel_20872\4255260337.py:4: WavFileWarning: Chunk (non-data) not understood, skipping it.
  input_audio3 = wavfile.read("photograph.wav")
C:\Users\1mack\AppData\Local\Temp\ipykernel_20872\4255260337.py:5: WavFileWarning: Chunk (non-data) not understood, skipping it.
  input_audio4 = wavfile.read("redroom.wav")


## Part 1. The broadcast

We will begin by setting up the broadcast process according to the following block diagram:

<img src="broadcast.png" width=700>

We'll do this in a couple different steps.

**Question 1.1 (1 point)**. First, design a lowpass filter to reduce the bandwidth of a signal to within the acceptable range of our radio stations.

In [30]:
def reduce_signal_bandwidth(signal):
    """Reduce the bandwidth of a signal to within a ~20 kHz band.
    
    Here, 20 kHz corresponds to the space a signal will take when considering
    both positive and negative frequencies, like we have seen during the 
    amplitude modulation lectures in class. 
    
    It is up to you to decide how to filter it. You may choose to use an 
    ideal brick wall filter, or a more realistic non-ideal filter (it doesn't
    need to be exactly 20 kHz wide, just something appropriate for the
    application).
    
    Args:
        signal (array[float]): A real-valued signal.
        
    Returns:
        array[float]: A signal that has been filtered so that its spectrum is 
        band-limited. 
    """
    frequencies = np.fft.fftfreq(signal)
    fourier_spectrum = np.fft.fft(signal)
    modified_spectrum = np.array([0 if(np.abs(frequencies[index]) > 10) else fourier_spectrum[index] for index in range(len(fourier_spectrum))])
    return np.fft.ifft(modified_spectrum)
    

**Question 1.2 (1 point)** Implement a function that performs sinusoidal amplitude modulation (with a $\cos$ carrier signal) of a signal into the frequency range of a desired radio station.

In [2]:
def sinusoidal_amplitude_modulation(signal, station_frequency, phase=0.0):
    """Modulate a signal up to the frequency of a desired AM radio station.
    
    The amplitude modulation should have the form:
       y(t) = x(t) * c(t) = x(t) * cos(2 * π * f_c * t + θ_c) 
    where f_c is the carrier (station) frequency, and θ_c is a phase.
    
    Args:
        signal (array[float]): A real-valued signal.
        station_frequency (int): The (carrier) frequency of the station, in kHz.
        phase (float): The phase of the carrier signal. 
        
    Returns:
        array[float]: The modulated signal.
    """
    omega = 2*np.pi*station_frequency
    modulated_signal = [signal[time]*np.cos(omega*time + phase) for time in range(len(signal))]
    return modulated_signal
    

**Question 1.3 (1 point)**. Your radio company will use frequency-division multiplexing (FDM) to send multiple signals over the same channel. Using the frequencies listed above, implement a broadcast (transmission) system that takes a set of signals as input, and outputs the full radio broadcast.

In [ ]:
def broadcast(signals):
    """Broadcast a set of signals at different frequencies to simulate 
    a set of radio stations.
    
    This function should:
      - reduce the bandwidth of each of the provided signals
      - modulate each one to the correct frequency range
      - perform FDM to generate a combined output signal to be broadcast
    
    Args:
        signals (list[array[float]]): A list of signals to be transmitted. 
             Assume that they are provided in order of increasing station frequency.
    
    Returns:
        array[float]: A single channel contain the frequency-division multiplexed
        set of signals.
    """
    return

## Part 2. A simple receiver

In order to test your broadcasting setup, let's implement a simple synchronous demodulation system.

**Question 2.1 (1 point)** Complete the function below to implement the synchronous demodulation system that we described in class (see the docstring below for details). 

In [1]:
def simple_receiver(broadcast, station_frequency, phase=0.0, bandpass=True):
    """Demodulate the radio broadcast and extract a desired station.
    
    This function should:
      - apply a bandpass filter (if specified) to the signal to extract
        only the parts of the spectrum in the desired station's frequency range
      - demodulate with a sinusoidal carrier (use cos) at the desired frequency
      - apply a low-pass filter with the appropriate gain to recover the signal
    
    Args:
        broadcast (array[float]): A radio broadcast consisting of five channels
            that have been FD multiplexed.
        station_frequency (int): The frequency (in kHz) of the radio station.
        phase (float): The phase of the demodulating signal.
        bandpass (bool): Whether or not to apply a bandpass filter before
            demodulating and filtering.
    
    Returns:
        array[float]: The signal of the radio station at the desired frequency.
    """
    return

**Question 2.2 (0.5 point)**. Test your demodulation function below to extract the 600 kHz station.

**Question 2.3 (0.5 point)**. Test your demodulation function with a non-zero phase; what happens to the output signal when the modulation and demodulation are not perfectly in sync?

**Question 2.4 (0.5 point)**. Set `bandpass=False` and test the demodulation; how does it sound?

**Question 2.5 (0.5 point)**. Try demodulating with an invalid station frequency (e.g., 590, 610, anything that sits in between our allotted values). Does this sound like you expect?

## Part 3: the superheterodyne receiver

In practice, AM radio is demultiplexed and demodulated by a system called the [superheterodyne receiver](https://en.wikipedia.org/wiki/Superheterodyne_receiver) (or superhet for short). Consider the (simplified) diagram below (there should in principle be some amplifiers in here, but we will ignore them):

<img src="superhet.png" width="800">

Here, $y(t)$ corresponds to the multiplexed signal that your broadcast outputs. This is put through a coarse tunable bandpass filter (this corresponds to you turning the radio dial), then mixed with a variable-frequency local oscillator to produce a signal centered at an intermediate frequency (IF) of 455 kHz. Said signal is then put through a fixed bandpass filter before being sent to the demodulator.

**Question 3.0 (0.5 point)**: Why do you think this kind of setup is used; why not just use a simple tunable filter from the get go? (You may need to do a little research on the history of the development of radio.)

**Question 3.1 (1 point)**: Implement the heterodyning process in the the graphic above. The function should take the multiplexed signal as input and a desired radio station frequency, and output the demodulated audio signal.

In [ ]:
def superhet_receiver(radio_broadcast, station_frequency):
    """Implement a simplified superheterodyne receiver to extract a desired station.
    
    This function should:
      - apply a bandpass filter to the signal to extract the correct 
        frequency regime 
      - use a local oscillator (frequency related to the bandpass) to yield an
        intermediate frequency signal at 455 kHz
      - apply a fixed bandpass filter and then demodulate to recover the station
    
    Args:
        broadcast (array[float]): A radio broadcast consisting of five channels
            that have been FD multiplexed.
        station_frequency (int): The frequency (in kHz) of the radio station.
    
    Returns:
        array[float]: The signal of the radio station at the desired frequency.
    """
    return

**Question 3.2 (0.5 point)**: Let's put everything together: below, use the broadcast function to create a signal, and then pass it to the superheterodyne receiver to play your favourite station. 